
*https://www.databricks.training/spark-ui-simulator/index.html*


## 1. Application Performance 
- The 5 most common performace problem (5's)
  - **1. Skew** - Imbalance in the size of the partitions.
  - **2. Spill** - Writing of Temp files to disk due to lack of memory.
  - **3. Storage** - A set of the problems directly related to how data is stored in disk.
  - **4. Shuffle** - Act of data moving between the executors.
  - **5. Serialization** - Distribution code of segments accross the cluster.

  ### One problem can cause others 
    - **Skew** can induce **Spill**
    - **Storage** issue can induce excess **shuffle** 
    - Incorrectly addressed **Shuffle** can exacerbate **Skew**
    - many of these prolems are present at same time

## 2. General Optimization 
* ## 1. Table Optimization
  - 1. Z Order
  - 2. Data Skipping
  - 3. Vacuum
  - 4. Partitioning
  - 5. Deletion Vectors
  - 6. Liquid Clustering
  - 7. Predictive Optimization

* ## 2. Cluster Tuning





# 1.1 ⚖️ Data Skew in Databricks

**Data skew** is a common performance bottleneck in distributed systems like Apache Spark and Databricks, where certain partitions contain significantly more data than others. This causes slow processing, inefficient resource use, and possible failures.

Data skew occurs when **some keys or partitions hold much more data** than others. It leads to unbalanced work distribution across executors.

### Common Causes

| Cause                      | Description                                                    |
| -------------------------- | -------------------------------------------------------------- |
| Skewed key values          | One or few keys dominate the dataset                           |
| NULLs                      | All NULLs grouped in one partition                             |
| Poor partitioning          | Uneven distribution due to default hash or manual partitioning |
| High-cardinality imbalance | Certain keys have much more data than others                   |

---

## ⚡ Why It Matters

| Issue                  | Impact                                                       |
| ---------------------- | ------------------------------------------------------------ |
| ⏱ Slow tasks           | Some partitions take significantly longer to compute         |
| 🛋 Shuffle overload    | Large shuffles stress network and disk                       |
| 📉 OOM errors          | Large partitions may cause out-of-memory failures            |
| 🥴 Underutilized nodes | Most workers stay idle waiting for one skewed task to finish |

---

## 📈 How to Detect Skew in Databricks

* **Spark UI** > Stages > Tasks:

  * Look for tasks with large data volume or unusually long durations
* **Skew logs**: Spark may emit warnings for skewed joins
* **File size patterns**: After write, some files may be much larger

---

## 🔧 Solutions to Handle Skew

### 1. **Salting the Key**

Add random digits to skewed keys to distribute them across partitions.

```python

from pyspark.sql.functions import concat, col, lit, rand

df = df.withColumn("salted_key", concat(col("join_key"), lit("_"), (rand() * 10).cast("int")))
```

---

### 2. **Broadcast Join**

Broadcast small dimension tables to avoid shuffling.

```python

from pyspark.sql.functions import broadcast

df_large.join(broadcast(df_small), "join_key")
```

---

### 3. **Repartitioning**

Repartition the DataFrame before join or shuffle.

```python

df.repartition("join_key")
```

---

### 4. **Skew Join Hints (DBR 7.3+)**

```sql

SELECT /*+ SKEW('large_df') */ *
FROM large_df JOIN small_df
ON large_df.key = small_df.key;
```

---

### 5. **Replace NULLs**

Avoid NULL clustering by replacing them.

```python

df.fillna({"join_key": "UNKNOWN"})
```

---

## ✅ Benefits of Skew Handling

| Benefit                 | Description                                     |
| ----------------------- | ----------------------------------------------- |
| ⚡ Faster Jobs           | Better distribution leads to quicker completion |
| 🚀 Fewer Failures       | Avoid OOM and executor crashes                  |
| 📊 Balanced Load        | All cores and workers used efficiently          |
| 🚗 Better Network Usage | Avoids unnecessary shuffle traffic              |

---

## ⚠ Limitations

| Limitation               | Description                                         |
| ------------------------ | --------------------------------------------------- |
| 🔧 Salting complexity    | Requires changes to logic before and after joins    |
| 🚫 Broadcast limitations | Only for small tables that fit in memory            |
| 🔄 Repartition cost      | Adds shuffle overhead if misused                    |
| ❌ Skew hint support      | Only in newer DBR versions                          |
| ⏳ Dynamic skew           | Skew may change over time; needs regular monitoring |

---

## 📒 Summary

| Topic     | Detail                                                  |
| --------- | ------------------------------------------------------- |
| Problem   | Uneven distribution of data across partitions           |
| Impact    | Slow jobs, high memory usage, shuffle cost              |
| Fixes     | Salting, Broadcast, Repartition, Skew Hints             |
| Benefits  | Improved speed, fewer failures, balanced resource usage |
| Tradeoffs | Added complexity, compute overhead, tuning required     |

---



# 1.2 📀 Spill in Databricks

In **Databricks** (based on Apache Spark), **spill** refers to writing intermediate data from memory to disk when memory is insufficient during operations like joins, sorts, or aggregations. Spill is a fallback mechanism to avoid job failure, but it impacts performance.

Spill happens when Spark **runs out of execution memory** and is forced to **offload data to disk** to continue processing.

### Typical Scenarios:

* Large **shuffle joins** or **groupBy** operations
* **Sort** operations on big datasets
* **Skewed partitions** or insufficient executor memory
* Multiple wide transformations in a pipeline

---

## 🔄 Types of Spill

| Type                | Description                                             |
| ------------------- | ------------------------------------------------------- |
| **Shuffle Spill**   | Data spilled during shuffle operations                  |
| **Sort Spill**      | Sort buffers that overflow memory are written to disk   |
| **Aggregate Spill** | Hash tables used in aggregation spill to disk when full |

---

## 📊 How Spark Manages Spill

Spark uses `UnifiedMemoryManager` to divide JVM memory into:

* **Execution memory** (joins, sorts, shuffles)
* **Storage memory** (caching, broadcast)

If execution memory exceeds its limit, **spill is triggered** to prevent `OutOfMemoryError`.

---

## 🔍 How to Detect Spill

* **Spark UI > Stages > Tasks Tab**: Look for high "Spilled Bytes"
* **Task skew**: Large variance in task durations
* **Logs**: Messages like `Spilling map output` or `Spilling sort buffer`

---

## ✅ Benefits of Spill

| Benefit                 | Description                                         |
| ----------------------- | --------------------------------------------------- |
| 🛡 Prevents Job Failure | Avoids out-of-memory crashes during execution       |
| 📀 Handles Large Data   | Processes datasets bigger than memory               |
| ✅ Graceful Degradation  | Keeps job running at the cost of speed              |
| 🚗 Supports Autoscaling | Complements cluster autoscaling for large workloads |

---

## ⚠ Limitations of Spill

| Limitation                     | Explanation                                         |
| ------------------------------ | --------------------------------------------------- |
| ⏳ Performance Overhead         | Disk I/O is slower than memory                      |
| ⛔ Frequent Spill = Poor Tuning | Could signal skew, bad joins, or too few partitions |
| 💳 Increased Storage Cost      | Extra reads/writes add cloud IOPS cost              |
| ❌ Not a True Optimization      | Spill avoids failure, not a performance tactic      |
| 🔬 Debug Complexity            | Root cause can be hidden without proper tools       |

---

## 🛠️ How to Minimize Spill

| Strategy                        | Action                                              |
| ------------------------------- | --------------------------------------------------- |
| ↗ Increase Executor Memory      | Allocate more memory per executor                   |
| 🧨 Broadcast Small Tables       | Avoid shuffle by replicating small dimension tables |
| 🔄 Repartition                  | Balance data before wide transformations            |
| 🌌 Fix Data Skew                | Use salting, skew join hints                        |
| 📦 Select Relevant Columns      | Avoid loading unnecessary data                      |
| ❄️ Use Adaptive Query Execution | Let Spark optimize joins and partitions dynamically |

---

## ⚙ Configuration Parameters

| Parameter                      | Purpose                                              |
| ------------------------------ | ---------------------------------------------------- |
| `spark.memory.fraction`        | % of JVM used for storage + execution (default: 0.6) |
| `spark.memory.storageFraction` | % of `memory.fraction` for caching (default: 0.5)    |
| `spark.sql.shuffle.partitions` | Controls shuffle output partitions (default: 200)    |

---

## 📒 Summary

| Feature      | Detail                                          |
| ------------ | ----------------------------------------------- |
| Definition   | Disk write fallback for out-of-memory scenarios |
| Triggered By | Joins, sorts, groupBy, shuffles, skewed data    |
| Pros         | Job safety, large data handling                 |
| Cons         | Slower performance, expensive I/O               |
| Detection    | Spark UI, logs, long task runtimes              |
| Fixes        | Tune memory, repartition, use AQE, avoid skew   |

---

# 1.3  📆 Storage Optimization in Databricks

Storage optimization in Databricks focuses on improving **performance**, **cost-efficiency**, and **scalability** by managing how data is stored, laid out, and read in Delta Lake or Parquet format. Key strategies include partitioning, clustering, compaction, and more.

---
## 🔧 Configuration Settings

| Setting                                             | Purpose                              |
| --------------------------------------------------- | ------------------------------------ |
| `spark.databricks.delta.optimizeWrite.enabled=true` | Merges small files during write      |
| `spark.databricks.delta.autoCompact.enabled=true`   | Triggers auto compaction after write |
| `spark.sql.files.maxPartitionBytes`                 | Controls partition size for reading  |

---

## 📅 Benefits of Storage Optimization

| Benefit               | Description                                   |
| --------------------- | --------------------------------------------- |
| ⚡ Faster Queries      | Reduces data scanned via pruning and skipping |
| 💸 Cost Efficiency    | Minimizes storage and compute cost            |
| 💡 Better Performance | Less memory, I/O, and shuffle overhead        |
| 🚀 Scalability        | Enables petabyte-scale operations             |

---

## ⚠ Limitations and Considerations

| Limitation                        | Explanation                                           |
| --------------------------------- | ----------------------------------------------------- |
| ⏳ OPTIMIZE is expensive           | Full-table rewrites can be costly                     |
| 📉 Partition skew                 | Too many or too few partitions can reduce performance |
| ❌ Z-ORDER is static               | Doesn't auto-adapt to workload changes                |
| 🔹 Temporary overhead             | Optimizations may temporarily use more disk           |
| ⚡ VACUUM removes time travel data | Irreversible deletion of old files                    |

---

## 📒 Summary

| Technique         | Purpose                     | Best Use Case                    | SQL Code                           |
| ----------------- | --------------------------- | -------------------------------- |--------------------------------
| Partitioning      | Skip large data segments    | Filters on date, region          | CREATE TABLE sales PARTITIONED BY (region, date); |
| Z-Ordering        | Improve file block locality | Joins/filters on key columns     | OPTIMIZE sales ZORDER BY (customer_id, date); |
| Liquid Clustering | Auto file layout            | Real-time or streaming workloads | ALTER TABLE events SET CLUSTER BY (event_type, date); |
| File Compaction   | Fix small files             | Frequent small writes            | OPTIMIZE table_name; |
| VACUUM            | Clean unused data           | Save storage cost                | VACUUM table_name RETAIN 168 HOURS; |
| Data Skipping     | Auto filter out blocks      | Enabled by default on Delta      | |

---



## 1.4 🔄 Databricks Shuffle Optimization – Complete Guide

Shuffle is a critical part of Spark and Databricks jobs, especially for wide transformations (like joins, groupBy, distinct, orderBy). However, it’s also expensive — it involves disk I/O, network I/O, and memory usage. Optimizing shuffle helps reduce cost, improve job performance, and prevent failures.

---

## ⚠️ What is Shuffle?

Shuffle happens when Spark redistributes data across partitions — usually triggered by:

* `groupByKey`, `reduceByKey`, `join`, `distinct`, `repartition`, `orderBy`, etc.

---

## ⚙️ Key Strategies for Shuffle Optimization

### 1. **Use Narrow Transformations Where Possible**

* Prefer `map`, `filter`, `flatMap`, `mapPartitions` — these don’t trigger shuffle.

### 2. **Use `reduceByKey` Instead of `groupByKey`**

* `reduceByKey` does a map-side combine before shuffle, reducing the amount of data shuffled.

### 3. **Broadcast Join Optimization**

Use **broadcast join** when one table is small enough (< 10 GB typically):

```python
from pyspark.sql.functions import broadcast
df1.join(broadcast(df2), "id")
```

* Reduces shuffle by sending the smaller dataset to all executors.

### 4. **Use Adaptive Query Execution (AQE)**

* AQE dynamically optimizes joins and shuffles at runtime.
  Enable in Spark config:

```python
spark.conf.set("spark.sql.adaptive.enabled", "true")
```

Key AQE features:

* Dynamic shuffle partitioning
* Skew join handling
* Dynamically switching join strategies

### 5. **Optimize Number of Shuffle Partitions**

Default: `spark.sql.shuffle.partitions = 200`

* Lower it for small jobs, raise for large ones.
* Tune based on dataset size, number of executors, and partitions.

### 6. **Handle Skewed Data**

Skew causes some tasks to process more data than others.

#### Strategies:

* Use **salting** for skewed keys
* Use **AQE's skew join handling**
* Filter hot keys into a separate branch

```python
spark.conf.set("spark.sql.adaptive.skewJoin.enabled", "true")
```

### 7. **Avoid Unnecessary `repartition` and `coalesce`**

* `repartition()` causes full shuffle — avoid unless needed.
* Use `coalesce()` to reduce partitions **without** full shuffle.

### 8. **Use Z-Ordering with Delta Tables**

* Helps skip files efficiently and improves read performance after shuffle-heavy operations.

```sql
OPTIMIZE table_name ZORDER BY (col1, col2)
```

---

## 🔍 Useful Configs for Shuffle Optimization

| Config Key                                      | Description                   | Recommended                                |
| ----------------------------------------------- | ----------------------------- | ------------------------------------------ |
| `spark.sql.shuffle.partitions`                  | Default shuffle partitions    | Tune based on workload (e.g. 200–1000)     |
| `spark.sql.adaptive.enabled`                    | Enables AQE                   | `true`                                     |
| `spark.sql.adaptive.skewJoin.enabled`           | Handles skew joins            | `true`                                     |
| `spark.sql.adaptive.coalescePartitions.enabled` | Dynamically reduce partitions | `true`                                     |
| `spark.sql.autoBroadcastJoinThreshold`          | Size threshold for broadcast  | Set based on memory (e.g., `10MB`, `50MB`) |

---

## ✅ Best Practices Checklist

* [x] Enable AQE
* [x] Broadcast small lookup tables
* [x] Tune `shuffle.partitions` per job size
* [x] Use `reduceByKey` instead of `groupByKey`
* [x] Avoid wide transformations unless necessary
* [x] Handle skewed keys carefully
* [x] Repartition only when really needed
* [x] Monitor job DAG & metrics from Spark UI


# 1.5 🔄 Serialization in Databricks

**Serialization** in Databricks refers to the process of converting **in-memory objects (like DataFrames, RDDs, or objects)** into a **byte stream** for transmission across nodes or writing to disk. It is essential for distributed computing and affects performance, memory, and network efficiency.

Serialization is:

> 🛠️ **Converting structured objects into a storable or transmittable format**, later reconstructed via deserialization.

### Used In:

* Shuffle (e.g., joins, groupBy)
* Caching or persisting
* Writing files (Delta, Parquet, JSON)
* UDF processing

---

## ⚙️ Serialization Formats in Databricks

| Format                 | Description                                                |
| ---------------------- | ---------------------------------------------------------- |
| **Java Serialization** | Default; flexible but slow and bulky                       |
| **Kryo Serialization** | Compact and fast; requires registration for custom classes |
| **Parquet/ORC/Avro**   | Used for storage; columnar and efficient                   |
| **Delta**              | Layer on top of Parquet with ACID metadata                 |

---

## 🧠 Where Serialization Happens

* During **shuffle** (joins, aggregations)
* **Caching** with `.cache()` or `.persist()`
* Writing to storage formats (Parquet, Delta, etc.)
* Passing objects in **UDFs** across executors

---

## ✅ Benefits of Serialization

| Benefit                       | Description                                      |
| ----------------------------- | ------------------------------------------------ |
| ⚡ Faster Network Transfer     | Smaller, compact byte streams transmitted faster |
| 📀 Efficient Storage          | Parquet/Kryo saves disk/memory space             |
| 🧠 Memory Management          | Kryo reduces object overhead in JVM heap         |
| 🚀 Better Shuffle Performance | Serialization reduces I/O bottlenecks            |

---

## ⚠️ Limitations of Serialization

| Limitation                    | Description                                          |
| ----------------------------- | ---------------------------------------------------- |
| ⏳ Java Serialization is slow  | Inefficient for large workloads                      |
| 🔧 Kryo requires setup        | Complex objects must be registered manually          |
| 👾 Opaque Format              | Not human-readable; needs deserialization to inspect |
| 🔄 (De)serialization Overhead | Adds latency for large/complex objects               |
| ⚡ Compatibility Risks         | Schema or code changes may break deserialization     |

---

## 🔧 Enabling Kryo in Databricks

```python

spark.conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
spark.conf.set("spark.kryo.registrationRequired", "false")
```

Register custom classes:

```python

spark.conf.set("spark.kryo.classesToRegister", "com.mycompany.MyClass")
```

---

## 📄 Summary

| Topic       | Detail                                                  |
| ----------- | ------------------------------------------------------- |
| Purpose     | Convert memory objects to bytes for I/O or distribution |
| Common Uses | Shuffle, caching, UDFs, file writes                     |
| Formats     | Java, Kryo, Parquet, Delta                              |
| Benefits    | Faster transfer, memory efficiency, better performance  |
| Limitations | Overhead, complexity, format compatibility              |

---


# 2.1.1 🔍 Z-Ordering
- Z-Ordering (or Z-Indexing) is a multi-dimensional clustering technique that reorders data within files to optimize data skipping during queries.

---
```sql

OPTIMIZE delta.`/path/to/table`
ZORDER BY (customer_id, order_date);

OPTIMIZE my_catalog.sales.orders
ZORDER BY (customer_id, order_date);
```
---

## 🔍 Behind the Scenes: What Happens?
  - Compaction: OPTIMIZE merges many small files into fewer large files.
  - Z-ordering: Within each new file, records are re-ordered using Z-curve logic based on chosen columns.
  - Data Skipping: During queries, Spark looks at min/max stats for each file → skips files that don’t match the filter.

## 🔎 Performance Gains
  - Query filtering on Z-ordered columns can skip up to 90–98% of files.
  - Major speedups observed for high-cardinality columns with selective filtering.

## 📌 When to Use Z-Ordering
| Use Case                                | Z-Ordering Recommended?              |
| --------------------------------------- | ------------------------------------ |
| Filtering on non-partitioned columns    | ✅ Yes                                |
| Frequent range scans                    | ✅ Yes                                |
| Filtering on booleans (low cardinality) | ❌ Not helpful                        |
| Full table scans                        | ❌ No benefit                         |
| Streaming workloads                     | ❌ Z-order not supported in streaming |
| ❌ Not for streaming tables             |	Must be batch   |

## 🧹 Schedule Z-Ordering
- You can schedule OPTIMIZE ZORDER in a notebook or workflow:
```sql

spark.sql("""
OPTIMIZE main.retail.transactions
ZORDER BY (customer_id, txn_date)
""")
```


# 2.1.2 🔍 Data Skipping in Databricks – Complete Guide

**Data Skipping** is a performance optimization feature in Databricks (Delta Lake) that allows Spark to **avoid scanning unnecessary files** during query execution by leveraging **metadata-level statistics**.

Data Skipping allows Delta Lake to skip reading files that **cannot possibly match the query filters**, based on per-file statistics like min/max values for each column.

> Instead of reading all files, Spark checks the Delta log metadata to see if a file's data range overlaps with the query filter.

---

## ⚙️ How Data Skipping Works

1. Delta Lake stores **file-level statistics** (minValue, maxValue, nullCount) in the Delta transaction log.
2. When a query with a filter is issued, Spark compares the filter condition with these statistics.
3. Files that **do not match the filter** are **skipped**, saving I/O and improving performance.

---

### 📊 Example

If a Delta table contains 1000 files and only 1 file contains rows for `date = '2023-01-01'`,

```sql
SELECT * FROM sales WHERE date = '2023-01-01'
```

will **only scan that 1 file**, skipping the other 999.

---

## 👁 Statistics Used

| Statistic   | Description                                |
| ----------- | ------------------------------------------ |
| `minValue`  | Minimum value of a column in a file        |
| `maxValue`  | Maximum value of a column in a file        |
| `nullCount` | Number of null values per column in a file |

These are automatically collected during write, update, and merge operations.

---

## 📊 How to Enable or Verify

* Data Skipping is **enabled by default** on all Delta tables.
* To verify:

```sql
DESCRIBE DETAIL delta.`/path/to/table`
```

* Or inspect Delta logs under `_delta_log/` for statistics in commit JSON files.

---

## 📅 Benefits of Data Skipping

| Benefit              | Description                                 |
| -------------------- | ------------------------------------------- |
| ⚡ Faster Queries     | Files not matching filters are not read     |
| 💸 Cost Efficient    | Saves compute, I/O, and memory resources    |
| 🔓 Easy to Use       | No manual config needed                     |
| ✨ Scales with ZORDER | Even more efficient with sorted file layout |

---

## ⚠ Limitations of Data Skipping

| Limitation                           | Explanation                                       |
| ------------------------------------ | ------------------------------------------------- |
| 🛠️ Delta-only                       | Only works on Delta tables                        |
| ❌ Not helpful for full scans         | Skipping is bypassed when scanning entire table   |
| 🥺 Inaccurate stats = no skipping    | Non-standard writes may skip stats collection     |
| 🚫 No benefit for unstructured types | Works best on integers, strings, dates            |
| ⚡ Update-heavy workloads             | Require re-OPTIMIZE to refresh effective skipping |

---

## 🛠️ How to Maximize Skipping

| Strategy                    | Action                                          |
| --------------------------- | ----------------------------------------------- |
| Partition wisely            | Use on low-cardinality, filter-heavy columns    |
| Use `OPTIMIZE ZORDER`       | Physically co-locate similar values             |
| Avoid UDF filters           | Use simple `WHERE column = 'value'` expressions |
| Stick to standard writes    | Use DataFrame APIs, `MERGE`, `COPY INTO`, etc.  |
| Periodically run `OPTIMIZE` | Rewrites files for efficient skipping           |

---

## 📒 Summary

| Feature     | Description                                          |
| ----------- | ---------------------------------------------------- |
| What        | Skips reading files that can't satisfy query filters |
| How         | Uses per-file column stats from Delta logs           |
| When        | Filtered queries on Delta tables                     |
| Benefits    | Faster, cheaper queries without manual tuning        |
| Limitations | Not for full scans, unstructured data, or non-Delta  |
| Tips        | Use ZORDER, avoid skew, use simple filters           |

---


# 2.1.3🧹 VACUUM in Databricks – Complete Guide

The `VACUUM` command in Databricks is used to clean up **old, unreferenced data files** from Delta Lake tables. It's essential for **managing storage**, **cost**, and **performance** in your Lakehouse environment.

In Delta Lake, every time you update, delete, or overwrite data, **new Parquet files** are created while the **old files are retained** to support **ACID transactions**, **time travel**, and **rollback**.

`VACUUM` removes these **obsolete files** that are **no longer needed** by Delta Lake.

```sql

VACUUM delta_table_name RETAIN 168 HOURS;
```

---

## 🧠 How It Works

* **Delta transaction log** tracks all file versions.
* `VACUUM` uses this log to find files **not referenced** by the current table state or by any time travel version within the retention period.
* It **physically deletes** these files from storage.

---

## 🕒 Retention Period

Delta Lake **defaults to 7 days (168 hours)** to ensure safe time travel and rollback.

You can **change this retention**:

```sql  

VACUUM delta_table_name RETAIN 24 HOURS; -- 1 day
```

But note:

> ⚠️ **Shorter than 7 days** requires:

```sql

SET spark.databricks.delta.retentionDurationCheck.enabled = false;
```

---

## ⚙️ Syntax

```sql

VACUUM [db_name.]table_name [RETAIN num HOURS]
```

* `RETAIN` is optional (default = 168 hours)
* You can also run it using PySpark:

```python

spark.sql("VACUUM my_table RETAIN 24 HOURS")
```

---

## 📆 Use Cases for `VACUUM`

| Use Case                  | Why Important?                               |
| ------------------------- | -------------------------------------------- |
| Frequent updates/deletes  | Old files accumulate, need cleanup           |
| Save storage cost         | Deleted/overwritten files still occupy space |
| Improve performance       | Reduces unnecessary file scan                |
| Time travel window passed | Files can now be safely deleted              |

---

## 🧪 Best Practices

* ✅ Schedule periodic `VACUUM` (e.g. daily or weekly) via workflows or jobs.
* ✅ Run after heavy overwrite, `MERGE`, or `DELETE` operations.
* ⚠️ Be cautious reducing retention if you rely on **time travel** or **rollback**.
* ✅ Combine with `OPTIMIZE` for storage + query performance.

---

## 📌 Related Configurations

| Config                                                  | Description                                    |
| ------------------------------------------------------- | ---------------------------------------------- |
| `spark.databricks.delta.retentionDurationCheck.enabled` | If `true`, blocks unsafe `VACUUM` under 7 days |
| `spark.databricks.delta.vacuum.parallelDelete.enabled`  | Enables parallel deletion for faster cleanup   |

---

## 🔭 Check Before Vacuuming

You can list files marked for deletion (dry run) with:

```python

from delta.tables import DeltaTable

deltaTable = DeltaTable.forName(spark, "my_table")
df = deltaTable.vacuum(retentionHours=24, dryRun=True)
df.show()
```

---

## 🚮 What VACUUM Does **NOT** Do

* ❌ It does **not** clean up transaction logs (`_delta_log`)
* ❌ It does **not** optimize small files — use `OPTIMIZE` for that
* ❌ It does **not** reclaim space from the metadata store (external systems)

---

## 📏 Example Workflow

```sql

-- Optional: disable retention check for aggressive cleanup
SET spark.databricks.delta.retentionDurationCheck.enabled = false;

-- Perform cleanup
VACUUM my_schema.my_table RETAIN 24 HOURS;
```

---

## ✅ Summary

| Feature           | Description                                |
| ----------------- | ------------------------------------------ |
| Purpose           | Remove obsolete Delta data files           |
| Default Retention | 168 hours (7 days)                         |
| Usage             | `VACUUM table [RETAIN x HOURS]`            |
| Danger            | Reducing retention can affect time travel  |
| Best Practice     | Schedule regularly + combine with OPTIMIZE |


# 2.1.4 📂 Partitioning in Databricks

**Partitioning** in Databricks (and Apache Spark/Delta Lake) refers to the **physical organization of data** into subdirectories based on one or more column values. It optimizes query performance, enables data skipping, and improves manageability for large datasets.

Partitioning is the technique of **dividing data into logical chunks** stored in separate folders by partition column(s).

Example:
```

/sales_data/date=2024-01-01/
/sales_data/date=2024-01-02/
```

Each partition folder contains data specific to the partition key value.

---

## 🛠️ How to Partition in Databricks

### SQL Table Creation

```sql

CREATE TABLE sales (
  id STRING,
  amount DOUBLE,
  date STRING
)
USING DELTA
PARTITIONED BY (date);
```

### DataFrame Write

```python

df.write.partitionBy("date").format("delta").save("/mnt/sales_data")
```

---

## ✅ Use Cases for Partitioning

| Use Case                 | Why It Helps                                                 |
| ------------------------ | ------------------------------------------------------------ |
| 🔍 Time-based analytics  | Partition by `date` or `month` for faster time range scans   |
| 📦 Geographic queries    | Partition by `region` or `state` to isolate reads            |
| 🔄 Incremental loads     | Enables efficient overwrite or update of specific partitions |
| 🧪 Filtering performance | Leverages partition pruning to skip irrelevant data          |
| ⏫ Large datasets         | Enables high parallelism for processing                      |

---

## 📅 Benefits of Partitioning

| Benefit                  | Description                                    |
| ------------------------ | ---------------------------------------------- |
| ⚡ Faster Queries         | Uses partition pruning to skip unneeded files  |
| 💸 Cost Savings          | Scans only relevant partitions, saving compute |
| 🧱 Efficient Maintenance | Easier to rewrite/delete small data segments   |
| 😄 Better Parallelism    | Improves multi-threaded read/write throughput  |

---

## ⚠️ Limitations of Partitioning

| Limitation                 | Explanation                                                    |
| -------------------------- | -------------------------------------------------------------- |
| 🚧 Too many partitions     | Can lead to small file issues (file explosion)                 |
| 📉 Skewed partitions       | Uneven partition sizes cause performance imbalance             |
| ❌ Not filter-effective     | Partition pruning works only with filters on partition columns |
| 🔄 Repartitioning overhead | Expensive to change partition structure after write            |
| ❎ Fixed schema             | Partitioning schema is hard to modify post-creation            |

---

## 🔍 Best Practices

* ✅ Use **low-cardinality**, frequently filtered columns like `date`, `state`
* ❌ Avoid high-cardinality fields (e.g., `user_id`, `transaction_id`)
* ✅ Monitor for skew and small file growth
* ✨ Combine with **Z-Ordering** or **Liquid Clustering** for better optimization

---

## 📄 Summary

| Category         | Detail                                                        |
| ---------------- | ------------------------------------------------------------- |
| Purpose          | Organize data into directories by key column(s)               |
| Example          | Partition by `date`, `region`, `state`                        |
| Benefits         | Faster queries, parallelism, data skipping                    |
| Limitations      | File explosion, skew, fixed layout                            |
| Use Cases        | Time-based reporting, regional analytics, incremental updates |
| Best Paired With | Delta Lake, OPTIMIZE, Z-ORDER, Liquid Clustering              |


# 2.1.5  Deletion Vectors in Delta Lake (Databricks)

**Deletion Vectors (DVs)** are a feature introduced in **Delta Lake 3.0+** that allow DELETE, UPDATE, and MERGE operations to be performed **without rewriting physical Parquet files**. Instead, DVs track deleted rows in compact metadata files, improving performance and reducing cost.

A **Deletion Vector** is a **bitmap index** that records **which rows in a Parquet file are logically deleted**, without physically removing the data.

### Instead of:

* Rewriting full files to apply deletes...

### Delta Lake:

* Writes a **sidecar deletion vector file** to track the deleted rows.

---

## 🧠 How It Works

1. Delta Lake stores data in Parquet files.
2. When a row is deleted (via DELETE, MERGE, etc.):

   * Delta logs the deleted row **position** in a deletion vector.
3. During reads:

   * Delta applies the deletion vector to **filter out** the deleted rows.

---

## ✅ Benefits of Deletion Vectors

| Benefit                        | Description                                                               |
| ------------------------------ | ------------------------------------------------------------------------- |
| ⚡ **Faster DML**               | DELETE, UPDATE, MERGE operations are much faster without rewriting files. |
| 💸 **Lower Cost**              | Reduces storage and compute usage.                                        |
| ♻️ **Efficient Compaction**    | Avoids file explosion due to frequent DML.                                |
| 📖 **Better Read Performance** | When combined with Photon or vectorized reads.                            |
| 💡 **Fine-Grained Deletes**    | Enables deleting specific rows in large files.                            |

---

## ❌ Limitations / Considerations

| Limitation                     | Description                                                                     |
| ------------------------------ | ------------------------------------------------------------------------------- |
| 🔍 **Extra Metadata**          | Slight metadata overhead to store DVs.                                          |
| 🦜 **Needs Cleanup**           | Deleted rows still exist physically — requires `OPTIMIZE` or `VACUUM` to purge. |
| ⚡ **Requires Delta Lake 3.0+** | Not supported in older versions.                                                |
| 🔢 **Read Impact**             | Slight read penalty (mitigated by Photon).                                      |
| ⚖️ **Compatibility**           | Not all tools can read DV-enabled tables.                                       |

---

## 📗 When Are DVs Used?

* Used during:

  * `DELETE`
  * `UPDATE`
  * `MERGE`
  * `COPY INTO`
* If:

  * Delta Lake version ≥ 3.0
  * Runtime supports DVs (e.g., DBR 13+)
  * Table has DVs enabled

---

## ⚙️ Enable Deletion Vectors

On table level:

```sql

ALTER TABLE my_table SET TBLPROPERTIES (
  'delta.enableDeletionVectors' = 'true'
);
```

Globally:

```python

spark.conf.set("spark.databricks.delta.properties.defaults.enableDeletionVectors", "true")
```

---

## 🔢 Check for Deletion Vectors

```sql

DESCRIBE DETAIL my_table;
```

Look for:

* `"deletionVectorEnabled": true`
* `"numDeletionVectors"` in the output

---

## 🧹 Cleanup (Physical Deletion)

Use `OPTIMIZE` to remove physically deleted rows:

```sql
OPTIMIZE my_table;
```

---

## 📅 Summary

| Feature        | Value                                             |
| -------------- | ------------------------------------------------- |
| Purpose        | Logical deletion without rewriting files          |
| Introduced In  | Delta Lake 3.0                                    |
| Benefits       | Faster DML, lower storage cost, avoids file churn |
| Requires       | Delta Lake 3.0+, DBR 13+                          |
| Cleanup Method | `OPTIMIZE`, `VACUUM`                              |
| Applies To     | DELETE, UPDATE, MERGE, COPY INTO                  |

---


# 2.1.6 🧠 What is Liquid Clustering? 
- Liquid clustering replaces table **partitioning and ZORDER** to simplify data layout decisions and optimize query performance. It provides the flexibility to redefine **clustering keys** without rewriting existing data, allowing data layout to evolve alongside analytic needs over time. Liquid clustering applies to both **Streaming Tables and Materialized Views**.

  - Tables often filtered by **high cardinality** columns.
  - Tables with significant **skew** in data distribution.
  - Tables that **grow quickly** and require maintenance and tuning effort.
  - Tables with **concurrent write requirements**.
  - Tables with **access patterns** that change over time.
  - Tables where a typical **partition key** could leave the table with too many or too few partitions.

## ✅ Benefits of Liquid Clustering

| Benefit                           | Description                                                                                             |
| --------------------------------- | ------------------------------------------------------------------------------------------------------- |
| ⚡ **Faster Queries**              | Improves read performance by reducing the amount of data scanned (especially for filter-heavy queries). |
| 👮 **Incremental Optimization**   | Works during normal writes; no need for expensive `OPTIMIZE ZORDER`.                                    |
| 💸 **Lower Cost**                 | Reduces the need for costly maintenance operations like full rewrites.                                  |
| 🔀 **Automatic Management**       | No manual triggers required once configured.                                                            |
| 🧱 **Better Small File Handling** | Maintains file size balance, reducing file explosion.                                                   |
| 🎯 **Query-Adaptive Clustering**  | Can adjust based on query workloads (in upcoming versions).    
---
## ⚖️ How to Enable Liquid Clustering

You define the clustering columns at table creation or alteration time.

```sql

CREATE TABLE my_table (
  id STRING,
  ts TIMESTAMP,
  value STRING
)
USING DELTA
CLUSTER BY (id, ts);
```

Or update an existing table:

```sql

ALTER TABLE my_table SET CLUSTER BY (id, ts);
```

Then Databricks **automatically applies clustering during writes**.
---
## 🚧 Limitations of Liquid Clustering

| Limitation                             | Explanation                                                                     |
| -------------------------------------- | ------------------------------------------------------------------------------- |
| 🔄 **Only for Delta Tables**           | Requires Delta Lake format                                                      |
| 🥺 **Still Evolving**                  | Not as mature as Z-Ordering; performance tuning is ongoing                      |
| 💸 **Less Control**                    | You don’t get precise control over clustering logic like with manual ZORDER     |
| 🗓️ **No Backward Optimization**       | It doesn’t re-cluster existing data immediately; only new/modified rows benefit |
| 📦 **Requires Unity Catalog (for GA)** | Generally available only with Unity Catalog on certain runtimes                 |
| ⚙️ **Not Available in All Runtimes**   | Requires specific Databricks runtime versions (DBR 13+ or later)                |
---
## 🧹 Liquid Clustering vs. Z-Ordering

| Feature      | Liquid Clustering                  | Z-Ordering                           |
| ------------ | ---------------------------------- | ------------------------------------ |
| Trigger      | Automatic                          | Manual (`OPTIMIZE ZORDER`)           |
| Cost         | Low (incremental)                  | High (full rewrite)                  |
| Maintenance  | Minimal                            | Requires periodic optimization       |
| Control      | Less granular                      | More customizable                    |
| File Rewrite | Partial/incremental                | Full                                 |
| Use Case     | Frequent writes, real-time updates | Mostly static or batch-loaded tables |

---




# 2.1.7 🚀 Predictive Optimization
- A fully automated, intelligent maintenance feature for **Unity Catalog managed Delta tables**. It uses **ML to schedule and execute key optimizations** when they are most beneficial, eliminating manual planning. **Serverless compting** should be enable.
  - OPTIMIZE (incremental clustering & compaction)
  - ANALYZE (gathering statistics)
  - VACUUM (removing stale files)
---
## 🛠️ How It Works
- Databricks monitors your query history.
- A ML model predicts the best data layout for performance:
   - File sizes
   - Data clustering (Z-order like)
   - Column access patterns
- The system automatically rewrites the table layout in the background.
---
## ✅ How to Enable Predictive Optimization
- It is enabled by default on Unity Catalog Delta tables (Premium and Enterprise tiers).
- **We can enable and disable at catalog level as well**
- **Predictive Optimization -**	ENABLE (inherited from METASTORE metastore_azure_eastus)
```sql 
 -- if you want to do via manual query 

 ALTER TABLE catalog.schema.table
SET TBLPROPERTIES (
  'delta.autoPredictiveOptimization' = 'true'
);

-- system table 
select * from system.storage.predictive_optimization_operations_history;
```
---
## Limitation 
- Predictive optimization is not available in all regions.
- For tables with deleted file retention duration (delta.deletedFileRetentionDuration) configured below the default of 7 days, predictive optimization runs VACUUM with retention duration as 7 days. See Configure data retention for time travel queries.
- Predictive optimization does not perform maintenance operations on the following tables:
	- Tables loaded to a workspace as Delta Sharing recipients.
	- External tables.

--- 
## 🆚 Predictive Optimization vs Manual OPTIMIZE
| Feature                | Predictive Optimization    | Manual OPTIMIZE       |
| ---------------------- | -------------------------- | --------------------- |
| Trigger                | Automatic (based on usage) | Manual or scheduled   |
| File Compaction        | ✅ Yes                      | ✅ Yes                 |
| Z-Ordering             | ✅ Similar behavior         | ✅ Explicit            |
| Resource Usage Control | ✅ Adaptive                 | ❌ Needs manual tuning |
| Granularity of Control | ❌ No direct control        | ✅ Full control        |
---




# 2.2 Databricks Cluster Optimization Best Practices

Cluster tuning in Databricks involves configuring clusters effectively to optimize job performance, reduce costs, and ensure reliability.

---

## ⚙️ Key Areas of Cluster Tuning

### 1. Cluster Sizing

* Right-size nodes based on workload (compute-intensive vs. memory-intensive).
* Suggested: Driver = 1 node, Executors = 2-10 nodes.
* Use autoscaling to handle variable loads:

```python
spark.conf.set("spark.dynamicAllocation.enabled", "true")
```

---

### 2. Cluster Type Selection

| Cluster Type   | Use Case                |
| -------------- | ----------------------- |
| All-purpose    | Development, notebooks  |
| Job cluster    | Production jobs         |
| SQL warehouse  | BI tools, dashboards    |
| Photon-enabled | Fast SQL execution      |
| Single-node    | Local jobs, ML training |

---

### 3. Memory and Executor Configuration

| Setting                        | Description                        |
| ------------------------------ | ---------------------------------- |
| `spark.executor.memory`        | Memory allocated to each executor  |
| `spark.executor.instances`     | Total number of executors          |
| `spark.executor.cores`         | Cores per executor                 |
| `spark.sql.shuffle.partitions` | Default = 200; tune based on joins |

## Use HDD VMs Instead of SSDs for Long-Running Jobs

- **Prefer HDD-backed VMs** (e.g., F8) over SSD variants (e.g., F8s), especially for **long-running streaming jobs**.
- Monitor the **cluster event logs** for disk-related events like disk expansion needs.
- Upgrade node size if disk expansions are frequent.
- Note: **HDD VMs do not support disk cache or Delta cache**.
---

### 4. Autoscaling

* Enable autoscaling clusters to grow/shrink based on demand.
* Example: `min_workers = 2`, `max_workers = 20`.

---

### 5. Caching and Persistence

* Use `.cache()` or `.persist()` if a DataFrame is reused multiple times **and** fits into memory.
* Avoid caching large datasets unnecessarily.

---

### 6. Adaptive Query Execution (AQE)

Enable AQE for dynamic query optimization:

```python
spark.conf.set("spark.sql.adaptive.enabled", "true")
```

Benefits:

* Optimizes join strategies
* Skew handling
* Dynamic partition tuning

---

### 7. Garbage Collection (GC) Tuning

* Use G1GC to reduce GC overhead:

```bash
--conf "spark.executor.extraJavaOptions=-XX:+UseG1GC"
--conf "spark.driver.extraJavaOptions=-XX:+UseG1GC"
```

* Monitor GC time in Spark UI -> Executors tab

---

### 8. Monitoring and Metrics

* Use Spark UI: Stages -> Tasks, Executors -> GC Time
* Tools: Ganglia, Datadog, CloudWatch
* Enable event logging for deeper analysis

---

### 9. Cost Optimization Tips

| Strategy                           | Benefit                      |
| ---------------------------------- | ---------------------------- |
| Use spot instances                 | Save 60-80% on compute costs |
| Enable auto-termination            | Avoid idle cluster charges   |
| Use serverless SQL warehouses      | Auto-scale for SQL queries   |
| Schedule OPTIMIZE/VACUUM off-hours | Reduces peak-time load       |

---

## 🔹 Summary

| Tuning Area      | Action                              |
| ---------------- | ----------------------------------- |
| Sizing           | Choose right executor & memory      |
| Cluster type     | Pick based on workload type         |
| Autoscaling      | Minimize idle and cost              |
| Memory config    | Tune `memory`, `cores`, `instances` |
| Adaptive queries | Enable AQE                          |
| GC tuning        | Use G1GC and monitor                |
| Cost control     | Use spot + serverless + scheduling  |

---


